In [ ]:
%load_ext autoreload
%autoreload 2

import os,sys
import mne
import utils  #my code
import json
import matplotlib.pyplot as plt
import numpy as np
import utils_preproc as upre
import globvars as gv

In [ ]:
#rawnames = ['S99_off_move']
rawnames = ['S03_off_hold']

rawnames = ['S01_off_hold', 'S01_off_move']

In [ ]:
sources_type = 'parcel_aal'
src_file_grouping_ind = 10
use_lfp_HFO = 0
src_type_to_use  = 'parcel_ICA'
crop_start = None
crop_end   = None

In [ ]:
#mods_to_load = ['LFP', 'src', 'EMG']
#mods_to_load = ['LFP', 'src', 'EMG', 'SSS','resample', 'FTraw']
mods_to_load = ['LFP', 'src', 'EMG', 'resample', 'afterICA']
mods_to_load = ['src', 'FTraw']
mods_to_load = ['src', 'LFP', 'EMG']
if use_lfp_HFO:
    mods_to_load += ['LFP_hires']

In [ ]:
raws_permod_both_sides = upre.loadRaws(rawnames,mods_to_load, sources_type, src_type_to_use, 
             src_file_grouping_ind)
raws_lfponly = []
raws_lfp_highres = []
raws_srconly = []
raws_emg_rectconv = []
for rawname_ in rawnames:  
    r = raws_permod_both_sides[rawname_]
    raws_lfponly       += [ r.get('LFP',None)  ]
    raws_lfp_highres   += [ r.get('LFP_hires',None)  ]
    raws_srconly       += [ r.get('src',None)  ]
    raws_emg_rectconv  += [ r.get('EMG',None)  ]
    

if crop_end is not None:
    allraws = raws_lfp_highres + raws_lfponly + raws_srconly + raws_emg_rectconv
    for raw_cur in allraws:
        raw_cur.crop(crop_start,crop_end)

In [ ]:
utils.getRawPerSide(raws_permod_both_sides['S01_off_hold']['LFP'], 'LFP', 'str', 
                    remove_anns = None, switch_sides_chinfo=True, switch_sides=True)

In [ ]:
rr = raws_permod_both_sides['S01_off_hold']['src']
rop = utils.changeRawInfoSides(rr)

In [ ]:
yy = 'fds'
uu = yy.replace('fd','sss')
yy, uu

In [ ]:
rr.ch_names

In [ ]:
rop.ch_names

In [ ]:
#help(rr.get_data )

In [ ]:
for 

In [ ]:
info_LFP= raws_permod_both_sides['S01_off_hold']['LFP'].info['chs'][0]
info_LFP

In [ ]:
info_LFP['ch_name'] += 'fd'

In [ ]:
info= raws_permod_both_sides['S01_off_hold']['src'].info

In [ ]:
info['chs'][0].keys()

In [ ]:
agree = 0
for chni,chinfo in enumerate(info['chs']):
    print(info['ch_names'][chni], chinfo['ch_name'],chinfo['loc'][:3] )
    if chinfo['ch_name'].find('msrcR') >= 0 and chinfo['loc'][0] >= 0:
        agree += 1
    if chinfo['ch_name'].find('msrcL') >= 0 and chinfo['loc'][0] < 0:
        agree += 1

In [ ]:
agree, len(info['chs'])

In [ ]:
info['chs'][334]

In [ ]:
info['chs'][336]

In [ ]:
info['chs'][400]

# Trying to merge

In [ ]:
rhold = raws_permod_both_sides['S01_off_hold']['FTraw']
rmove = raws_permod_both_sides['S01_off_move']['FTraw']

In [ ]:
lthold = len(rhold.times)
ltmove = len(rmove.times)
dur = 5
nb = int(dur*rhold.info['sfreq'])
dth,tsh = rhold[:,-nb:]
dtm,tsm = rmove[:,:nb]

dthi,tshi = rhold[:,:nb]
dtmi,tsmi = rmove[:,-nb:]

In [ ]:
dth.shape

In [ ]:
chis = range(50)
N = len(chis)
ww = 5; hh=2
fig,axs = plt.subplots(N,2,figsize=(2*ww,N*hh))

for chii in range(N):
    
    chi = chis[chii]
    chnh = rhold.ch_names[chi]
    chnm = rmove.ch_names[chi]
    assert chnh == chnm
    
    ax = axs[chii,0]
    ax.plot(tsh,dth[chi],label='hold')
    ax.plot(rhold.times[-1]+tsm,dtm[chi],label='move')
    ax.legend()
    ax.set_title(chnh)

    ax = axs[chii,1]
    ax.plot(rmove.times[-1] + tshi,dthi[chi],label='hold')
    ax.plot(tsmi,dtmi[chi],label='move')
    ax.legend()
    
plt.tight_layout()
plt.savefig('merge_{}_and_{}.png'.format(rawnames[0],rawnames[1]))
plt.close()

In [ ]:
import gc; gc.collect()

In [ ]:
help(raw_FT.filter)

In [ ]:
sfreq = int(raws_srconly[0].info['sfreq'])
sfreq_highres = raws_lfp_highres[0].info['sfreq']
sfreq_highres = int(sfreq_highres)

In [ ]:
raw_FT = raws_permod_both_sides[rawnames[0]]['FTraw']
raw_FT_meg = raw_FT.copy()
raw_FT_meg.pick_types(meg=True)

In [ ]:
badchlist = upre.loadBadChannelList(rawname_,raw_FT_meg.ch_names)
raw_FT_meg.info['bads'] = badchlist


In [ ]:
raw_FT_meg.drop_channels(badchlist)

In [ ]:
r = utils.findRawArtifacts(raw_FT_meg, thr_use_mean = 1, data_mod='MEG')

In [ ]:
anns, cvlps = r

In [ ]:
raw_FT_meg.set_annotations(anns)

In [ ]:
%matplotlib qt

In [ ]:
#MEG1723 -- squid jump around 460s

In [ ]:
raw_FT_meg.info['chs']

In [ ]:
raw_FT_meg.plot();

In [ ]:
raw_FT_lfp.ch_names

In [ ]:
raw_FT_lfp = raw_FT.copy()
chns = np.array(raw_FT.ch_names)[ mne.pick_channels_regexp(raw_FT.ch_names,'LFP*') ]
raw_FT_lfp.pick_channels(chns)

In [ ]:
utils.findRawArtifacts(raw_FT_lfp, thr_use_mean = 1, data_mod='LFP')

In [ ]:
raw_FT_lfp.info['chs']

In [ ]:
raw_src = raws_permod_both_sides[rawnames[0]]['src']

In [ ]:
#raw_src.plot();

In [ ]:
%matplotlib qt

In [ ]:
#S01_off_hold 355.421875,1.0859375,BAD_MEGR

In [ ]:
raw_afterICA = raws_permod_both_sides[rawnames[0]]['afterICA']
raw_afterICA_meg = raw_afterICA.copy()
raw_afterICA_meg.pick_types(meg=True)

In [ ]:
raw_afterICA_meg.plot()

In [ ]:
raw_SSS = raws_permod_both_sides[rawnames[0]]['SSS']
raw_SSS_meg = raw_SSS.copy()
raw_SSS_meg.pick_types(meg=True)

In [ ]:
%matplotlib qt

In [ ]:
#raw_SSS_meg.plot(butterfly=1);
raw_SSS_meg.plot();

In [ ]:
raw_resample = raws_permod_both_sides[rawnames[0]]['resample']
raw_resample_meg = raw_resample.copy()
raw_resample_meg.pick_types(meg=True)

In [ ]:
raw_resample.info['chs']

In [ ]:
#raw_SSS_meg.plot(butterfly=1);
raw_resample_meg.plot();

In [ ]:
import gc; gc.collect()

In [ ]:
sources_type = 'parcel_aal'
src_file_grouping_ind = 10
src_grouping = 0 
crop_start = None
crop_end   = None
use_main_LFP_chan = 0
use_main_tremorside = 1
maintremside = 'left'
mts_letter = maintremside[0].upper()
data_modalities = ['LFP', 'msrc']

only_load_data = 0

In [ ]:
raws_permod_both_sides

In [ ]:
opside_let = utils.getOppositeSideStr(mts_letter)
srcraw = raws_permod['msrc'][opside_let]

In [ ]:
srcraw.ch_names

In [ ]:
supp_info_fname = rawname_ + '_supp_info.npz'
supp_info_fname_full = os.path.join(gv.data_dir,supp_info_fname)
supp_info = np.load(supp_info_fname_full, allow_pickle=1)['supp_info'][()]

In [ ]:
raws_srconly

In [ ]:
band = supp_info['trem_burst_meg_freq'] - 4, supp_info['trem_burst_meg_freq'] + 4

In [ ]:
srcdat.shape

In [ ]:
%matplotlib qt

In [ ]:
srcraw.plot();

In [ ]:
srcgroups = rec_info['srcgroups_dict'][()]['all_raw']
inds_per_roi = {}
srcraw = raws_srconly[0]
srcraw = srcraw.copy()
#srcraw.filter(l_freq=band[0],h_freq=band[1])

ch_names = srcraw.ch_names
labels_to_show = supp_info['roi_labels_with_tremor']
#labels_to_show = roi_labels_cur
for roi_lbl in labels_to_show:
    lbl_ind = roi_labels_cur.index(roi_lbl)
    #src_inds = np.where(srcgroups==lbl_ind)[0]
    side = roi_lbl[-1].upper()
    src_inds = mne.pick_channels_regexp(ch_names,'msrc{}_0_{}_c.*'.format(side,lbl_ind))
    
    inds_per_roi[roi_lbl] = src_inds
    print(roi_lbl, len(src_inds), type(src_inds))
    if len(src_inds) == 0:
        continue
    srcdat,times = srcraw[src_inds]
    #mean_srcdat = np.mean(srcdat,axis=0)
    #plt.plot(times,mean_srcdat)
    nsrcs = srcdat.shape[0]
    fig,axs = plt.subplots(nrows=nsrcs, ncols=1, sharex='col', figsize=(5,nsrcs*1.5))
    if not isinstance(axs,np.ndarray):
        axs = np.array([axs])
    for srci in range(nsrcs):
        ax = axs[srci]
        #plt.figure(figsize=(5,1))
        ax.plot(times,srcdat[srci])
        ax.set_title('{}:{}_{}_{}'.format(lbl_ind,roi_lbl, srci, srcraw.ch_names[src_inds[srci] ]) )
    plt.tight_layout()
    plt.show()
#     plt.title(roi_lbl)
#     plt.show()
    

In [ ]:
src_inds

In [ ]:
list(rec_info.keys())